In [9]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [58]:
end_time = datetime.now()
start_time = end_time - timedelta(days=2)
btc_price = vbt.YFData.download(
    ["BTC-USD", "ETH-USD"],
    missing_index="drop",
    start=start_time,
    end=end_time,
    interval="1m",
).get("Close")
btc_price

symbol,BTC-USD,ETH-USD
Datetime,,
2023-12-30 11:08:00+00:00,41869.164062,2289.531982
2023-12-30 11:09:00+00:00,41872.617188,2288.994385
2023-12-30 11:10:00+00:00,41876.777344,2289.540283
2023-12-30 11:11:00+00:00,41896.976562,2291.386230
2023-12-30 11:12:00+00:00,41907.988281,2291.145996
...,...,...
2024-01-01 11:00:00+00:00,42698.199219,2302.207275
2024-01-01 11:01:00+00:00,42703.140625,2302.366455
2024-01-01 11:02:00+00:00,42713.035156,2302.062744


## Custom indicators

In [59]:
def custom_indicator(close, rsi_window, ma_window):
    close_5m = close.resample("5T").last()
    rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
    rsi, _ = rsi.align(
        close, broadcast_axis=0, method="ffill", join="right", axis=0
    )
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    close = close.to_numpy()
    rsi = rsi.to_numpy()

    trend = np.where(rsi > 70, -1, 0)
    trend = np.where((rsi < 30) & (close < ma), 1, trend)

    return trend

In [60]:
ind = vbt.IndicatorFactory(
    class_name="Combination",
    short_name="comb",
    input_names=["close"],
    param_names=["rsi_window", "ma_window"],
    output_names=["trend"],
).from_apply_func(custom_indicator, rsi_window=14, ma_window=50, keep_pd=True)

In [61]:
res = ind.run(btc_price, rsi_window=21, ma_window=30)
print(res.trend.to_string())
# print(res.rsi_5m)
entries = res.trend == 1.0
exits = res.trend == -1.0
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.stats()
# fig = res.value.plot()

comb_rsi_window                21        
comb_ma_window                 30        
symbol                    BTC-USD ETH-USD
Datetime                                 
2023-12-30 11:08:00+00:00       0       0
2023-12-30 11:09:00+00:00       0       0
2023-12-30 11:10:00+00:00       0       0
2023-12-30 11:11:00+00:00       0       0
2023-12-30 11:12:00+00:00       0       0
2023-12-30 11:13:00+00:00       0       0
2023-12-30 11:14:00+00:00       0       0
2023-12-30 11:15:00+00:00       0       0
2023-12-30 11:16:00+00:00       0       0
2023-12-30 11:17:00+00:00       0       0
2023-12-30 11:18:00+00:00       0       0
2023-12-30 11:19:00+00:00       0       0
2023-12-30 11:20:00+00:00       0       0
2023-12-30 11:21:00+00:00       0       0
2023-12-30 11:22:00+00:00       0       0
2023-12-30 11:23:00+00:00       0       0
2023-12-30 11:24:00+00:00       0       0
2023-12-30 11:25:00+00:00       0       0
2023-12-30 11:26:00+00:00       0       0
2023-12-30 11:27:00+00:00       0 

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/conda/envs/default/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/tmp/ipykernel_49482/3817869639.py:7: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7f0894117010>. Pass column to select a single column/group.



Start                         2023-12-30 11:08:00+00:00
End                           2024-01-01 11:04:00+00:00
Period                                             2840
Start Value                                       100.0
End Value                                     99.642777
Total Return [%]                              -0.357223
Benchmark Return [%]                           1.316551
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               1.888177
Max Drawdown Duration                            1502.5
Total Trades                                        3.5
Total Closed Trades                                 3.5
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                       37.5
Best Trade [%]                                  0.01445
Worst Trade [%]                               -0

In [87]:
pf.trades.records

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
0,0,0,0.002370,702,42187.773438,0.0,800,42261.851562,0.0,0.175591,0.001756,0,1,0
1,1,0,0.002374,907,42202.945312,0.0,1028,42285.562500,0.0,0.196105,0.001958,0,1,1
2,2,0,0.002358,1457,42558.062500,0.0,2242,42471.402344,0.0,-0.204385,-0.002036,0,1,2
3,3,0,0.002359,2380,42462.371094,0.0,2626,42486.882812,0.0,0.057822,0.000577,0,1,3
4,4,1,0.043586,697,2294.309570,0.0,1195,2290.435547,0.0,-0.168854,-0.001689,0,1,4
5,5,1,0.043345,1594,2303.177490,0.0,2218,2289.216064,0.0,-0.605158,-0.006062,0,1,5
6,6,1,0.043339,2375,2289.511230,0.0,2626,2285.690918,0.0,-0.165570,-0.001669,0,1,6
